# Optimisation via Simulation Tutorial

By Christine S.M Currie and Tom Monks

In [1]:
import numpy as np

## Models

To do.

* Need to rename some of the functions so that it fits
* Need to adapt Christine's Law inventory model

In [3]:
from ovs.toy_models import (custom_guass_bandit_problem,
                            BanditCasino, 
                            guassian_bandit_sequence)

## Procedure **KN**

To run Kim and Nelson's R&S procedure KN, create an instance of `ovs.indifference_zone.KN`

An object of type KN takes the following parameters:

* **model** - a simulation model
* **n_designs** - int, the number of competing designs to compare
* **delta** - float, the indifference zone
* **alpha** - float, $PCS = 1-\alpha$ (default=0.05)
* **n_0** - int, $n_0$ the number of initial replications (default=2)

In [4]:
from ovs.indifference_zone import KN

In [24]:
designs = guassian_bandit_sequence(1, 11)
environment = BanditCasino(designs)

kn = KN(model=environment, 
        n_designs=len(designs), 
        delta=0.1, 
        alpha=0.05, 
        n_0=5)

In [26]:
best_design = kn.solve()
print('best design\t{0}'.format(best_design))
print('allocations\t{0}'.format(kn._actions))
print('total reps\t{0}'.format(kn._actions.sum()))
print('means\t\t{0}'.format(kn._means))

best design	[9]
allocations	[ 23  24  69  31  51  29  30  52 464 470]
total reps	1243
means		[0.96 1.8  2.93 3.77 5.   6.03 7.28 8.22 9.   9.93]


## Optimal Computing Budget Allocation (OCBA)

An object of type OCBA takes the following parameters:

* **model** - a simulation model
* **n_designs** - int, the number of competing designs to compare
* **budget** - int, the total number of replications to allocate across designs
* **delta** - int, the incremental amount of replications to allocate at each round
* **n_0** - int, $n_0$ the number of initial replications (default=5)
* **min** - bool, True if minimisation; False if maximisation (default=False)

In [7]:
from ovs.fixed_budget import OCBA

In [11]:
designs = guassian_bandit_sequence(1, 11)
environment = BanditCasino(designs)

ocba = OCBA(model=environment, 
            n_designs=len(designs), 
            budget=300, 
            delta=10, 
            n_0=5, 
            min=False)

call the `solve()` method to run the optimisation

In [19]:
results = ocba.solve()
print('best design:\t{}'.format(results))
print('allocations:\t{}'.format(ocba._allocations))
print('total reps:\t{}'.format(ocba._allocations.sum()))

np.set_printoptions(precision=2)
print('means:\t\t{0}'.format(ocba._means))
print('vars:\t\t{0}'.format(ocba._vars))

best design:	9
allocations:	[ 40  40  40  40  40  40  40  40  61 269]
total reps:	650
means:		[ -1.    -2.    -3.05  -3.74  -4.8   -6.14  -6.98  -7.87  -9.21 -10.05]
vars:		[0.9  1.14 0.75 1.59 0.95 0.65 1.07 1.18 0.94 1.1 ]
